
# Import and model

In [76]:
from torchvision.transforms import Compose, Resize, Normalize, ToTensor
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
import numpy as np
import torch
import pickle
import os
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from sklearn.metrics import auc
from ranx import Run, fuse

In [77]:
def normalize_data(data):
    # Kiểm tra nếu data là một vector 1 chiều
    if data.ndim == 1:
        normalized_data = data / np.linalg.norm(data)
    else:
        # Chuẩn hóa dữ liệu nếu là mảng 2 chiều
        normalized_data = data / np.linalg.norm(data, axis=1, keepdims=True)
    
    return normalized_data

In [78]:
def load_file_lines(file_path):
    """Đọc toàn bộ dòng từ một file và trả về danh sách các dòng."""
    try:
        with open(file_path, 'r') as f:
            return [line.strip() for line in f]
    except FileNotFoundError:
        print(f"File {file_path} không tồn tại!")
        return []

In [79]:
def precision(true_positives, retrieved):
  return true_positives / retrieved if retrieved != 0 else 0

def recall(true_positives, relevant):
  return true_positives / relevant if relevant != 0 else 0

def f1_score(precision, recall):
  return 2 * precision * recall / (precision + recall)

def average_precision(relevant_items, retrieved_items):
  '''
  relevant_items: list các ID mà thực sự là đúng
  retrieved_items: list các ID được truy vấn trả về
  '''
  relevant_items = set(relevant_items)
  retrieved = 0
  true_positives = 0
  ap = 0

  for i, item in enumerate(retrieved_items):
    retrieved += 1
    if item in relevant_items:
      true_positives += 1
      ap += precision(true_positives, retrieved)

  return ap / len(relevant_items) if relevant_items else 0

def mean_average_precision(queries):
  aps = [average_precision(q[0], q[1]) for q in queries]
  return np.mean(aps) if aps else 0

# Hàm vẽ Precision-Recall Curve
def plot_precision_recall_curve(queries, tittle='Precision-Recall Curve'):
    plt.figure(figsize=(10, 6))

    for idx, (relevant_items, retrieved_items) in enumerate(queries):
        relevant_items = set(relevant_items)
        precision_vals = []
        recall_vals = []
        true_positives = 0

        # Tính Precision và Recall cho từng ngưỡng
        for i, item in enumerate(retrieved_items):
            if item in relevant_items:
                true_positives += 1
            p = precision(true_positives, i + 1)
            r = recall(true_positives, len(relevant_items))
            precision_vals.append(p)
            recall_vals.append(r)

        auc_score = auc(recall_vals, precision_vals)

        # Vẽ đường Precision-Recall cho từng truy vấn
        plt.plot(recall_vals, precision_vals, label=f'AUC = {auc_score:.2f}')

    # Cấu hình biểu đồ
    plt.title(tittle)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.legend()
    plt.grid()
    plt.show()

# Parameters

In [80]:
gt_path = r'C:\Retrieval System\data\paris_120310_gt'
data_path = r'C:\Retrieval System\data\paris'
features_path = r'C:\Retrieval System\features\features_paris_clip_3'

In [81]:
k = 6392
retrain = True
top_rerank = 10

In [82]:
model = CLIPModel.from_pretrained('openai/clip-vit-base-patch32')
processor = CLIPProcessor.from_pretrained('openai/clip-vit-base-patch32')
if retrain:
    model = torch.load(r'C:\Retrieval System\model\paris_clip_3.pth', map_location=torch.device('cpu'))

model.eval()

transform = Compose([
    Resize((224, 224)),
    ToTensor(),
    Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))
])

c:\Users\DELL\miniconda3\envs\CS336\Lib\site-packages\huggingface_hub\file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


# Get input features

In [83]:
def process_queries(query_file, device='cpu'):
    """
    Đọc file query, cắt ảnh theo bounding box và extract feature theo từng bb.
    
    :param query_file: Đường dẫn đến file query.txt
    :param image_folder: Thư mục chứa ảnh
    """
    with open(query_file, 'r') as file:
        features = []
        for line in file:
            # Tách dòng query thành các phần tử
            parts = line.strip().split()
            if len(parts) != 5:
                continue
            
            image_name = parts[0] + '.jpg'  # Tên file ảnh
            x1, y1, x2, y2 = map(float, parts[1:])  # Bounding box
            
            # Đường dẫn đầy đủ đến ảnh
            image_path = os.path.join(data_path, image_name.split('_')[1], image_name)
            
            # Kiểm tra ảnh tồn tại
            if not os.path.exists(image_path):
                print(f"Ảnh {image_name} không tồn tại trong thư mục {os.path.join(data_path, image_name.split('_')[1])}.")
                continue
            
            # Mở ảnh và cắt theo bounding box
            with Image.open(image_path).convert('RGB') as img:
                cropped_img = img.crop((x1, y1, x2, y2))  # Cắt ảnh
                # Extract feature của ảnh crop
                cropped_img = transform(cropped_img).unsqueeze(0).to(device)
                with torch.no_grad():
                    feature = model.get_image_features(cropped_img)
                feature = feature.squeeze().cpu().numpy()
                features.append(feature)
        
        return features

In [84]:
query = []
map_qr = []
for file in os.listdir(gt_path):
    if 'query' in file:
        # print(file)
        map_qr.append(file.split('.')[0])
        query_file = os.path.join(gt_path, file)
        features = process_queries(query_file=query_file)
        if len(features) == 1:
            query.append(features[0])

query = np.array(query)
map_qr = [name[:-6] if name.endswith("_query") else name for name in map_qr]
print(map_qr)
query.shape

['defense_1', 'defense_2', 'defense_3', 'defense_4', 'defense_5', 'eiffel_1', 'eiffel_2', 'eiffel_3', 'eiffel_4', 'eiffel_5', 'invalides_1', 'invalides_2', 'invalides_3', 'invalides_4', 'invalides_5', 'louvre_1', 'louvre_2', 'louvre_3', 'louvre_4', 'louvre_5', 'moulinrouge_1', 'moulinrouge_2', 'moulinrouge_3', 'moulinrouge_4', 'moulinrouge_5', 'museedorsay_1', 'museedorsay_2', 'museedorsay_3', 'museedorsay_4', 'museedorsay_5', 'notredame_1', 'notredame_2', 'notredame_3', 'notredame_4', 'notredame_5', 'pantheon_1', 'pantheon_2', 'pantheon_3', 'pantheon_4', 'pantheon_5', 'pompidou_1', 'pompidou_2', 'pompidou_3', 'pompidou_4', 'pompidou_5', 'sacrecoeur_1', 'sacrecoeur_2', 'sacrecoeur_3', 'sacrecoeur_4', 'sacrecoeur_5', 'triomphe_1', 'triomphe_2', 'triomphe_3', 'triomphe_4', 'triomphe_5']


(55, 512)

# Load database

In [85]:
database = []
map_db = []
for file in os.listdir(features_path):
    file_path = os.path.join(features_path, file)

    with open(file_path, "rb") as f:
        image_features_data = pickle.load(f) # 1 list, các phần tử là dict với image_name và features

        for item in image_features_data:
            database.append(item['features'])
            map_db.append(item['image_name'].replace('.jpg',''))

database = np.array(database)
database.shape

(6392, 512)

# Retrieval

In [86]:
def retrieval_top_k(query, database, k=5):
    similarity_matrix = cosine_similarity(query, database)

    sorted_similarity_indices = np.argsort(similarity_matrix, axis=1)[:, ::-1] # Sắp xếp giảm dần

    # # Sắp xếp lại map_db theo thứ tự của similarity_matrix
    # sorted_map_db = []
    # for i in range(similarity_matrix.shape[0]):
    #     sorted_map_db.append([map_db[j] for j in sorted_similarity_indices[i]])


    top_k = []
    top_k_similarities = []

    for i in range(sorted_similarity_indices.shape[0]):
        top_k_indices = sorted_similarity_indices[i][:k]
        top_k.append([map_db[j] for j in top_k_indices])  # Lấy các đối tượng tương ứng
        top_k_similarities.append(similarity_matrix[i][top_k_indices])  # Lấy cosine similarity

    return top_k, top_k_similarities

# retrieval_top_k(query=query, database=database, k=2)

In [87]:
def retrieval_threshold(query, database, map_db, threshold=0.8):
    # Tính toán ma trận cosine similarity
    similarity_matrix = cosine_similarity(query, database)

    # Sắp xếp các chỉ số giảm dần của cosine similarity
    sorted_similarity_indices = np.argsort(similarity_matrix, axis=1)[:, ::-1]

    # Khởi tạo danh sách trả về
    filtered_map_db = []
    filtered_similarities = []

    # Duyệt qua từng query và lọc các kết quả có cosine similarity > threshold
    for i in range(similarity_matrix.shape[0]):
        # Lấy các chỉ số có độ tương đồng lớn hơn threshold
        valid_indices = sorted_similarity_indices[i][similarity_matrix[i, sorted_similarity_indices[i]] > threshold]
        filtered_map_db.append([map_db[j] for j in valid_indices])  # Lấy các đối tượng tương ứng
        filtered_similarities.append(similarity_matrix[i, valid_indices])  # Lấy cosine similarity

    return filtered_map_db, filtered_similarities

# retrieval_threshold(query=query, database=database, threshold=0.8)

In [88]:
rank_list, similarity_matrix = retrieval_top_k(query=query, database=database, k=k)

# Evaluate

In [89]:
ground_truths = []
for query_name in map_qr:
    file_ok = os.path.join(gt_path, f"{query_name}_ok.txt")
    file_good = os.path.join(gt_path, f"{query_name}_good.txt")

    # Đọc dòng từ từng file
    lines_ok = load_file_lines(file_ok)
    lines_good = load_file_lines(file_good)

    # Gộp nội dung 2 file thành 1 danh sách
    merged_list = lines_good + lines_ok

    ground_truths.append(merged_list)

In [90]:
queries = []
for i in range(len(rank_list)):
    relevant_items = ground_truths[i]
    retrieved_items = rank_list[i]
    queries.append((relevant_items, retrieved_items))
    # plot_precision_recall_curve(queries=[(relevant_items, retrieved_items)], tittle=f'Precision-Recall Curve of query {i+1}')
    # print(average_precision(relevant_items=relevant_items, retrieved_items=retrieved_items))

In [91]:
mean_average_precision(queries=queries)

0.7721015140756843

# Re-ranking

In [92]:
def merge_results_by_ranx(list_rank_list, list_similarity_matrix, method='gmnz'): # Merge kết quả của từng query.
  # Chuyển đổi dữ liệu sang dạng dictionary
  runs = []
  for i, (res, sims) in enumerate(zip(list_rank_list, list_similarity_matrix)):
      query_id = f"query_0"
      run_data = {str(doc_id): score for doc_id, score in zip(res, sims)}
      run = Run({query_id: run_data})
      runs.append(run)
  # Hợp nhất kết quả
  fused_run = fuse(runs, method=method)  # Có thể điều chỉnh weights nếu cần

  fused_ranking_results = {}

  for query_id, docs in fused_run.to_dict().items():
      fused_ranking_results[query_id] = list(docs.keys())  # Lấy danh sách document IDs (không cần điểm số)

  return fused_ranking_results['query_0']

In [93]:
rank_list[0][:top_rerank]

['paris_defense_000605',
 'paris_defense_000592',
 'paris_defense_000090',
 'paris_defense_000503',
 'paris_defense_000207',
 'paris_defense_000094',
 'paris_defense_000054',
 'paris_defense_000000',
 'paris_defense_000408',
 'paris_defense_000259']